In [1]:
import pandas as pd
from tensorflow.python.keras.optimizer_v1 import rmsprop

In [14]:
df = pd.read_csv('datasets/spam.csv', encoding='latin-1')

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [15]:
df=df[['v1','v2']]

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.shape

(5572, 2)

In [7]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [8]:
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
df = df.rename(columns={'v1': 'label', 'v2': 'message'})

In [17]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# using TF-IDF (Term Frequency-Inverse Document Frequency):

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
x=df['message']
y=df['label']

In [20]:
x.head(2)

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
Name: message, dtype: object

In [22]:
x=x.str.lower().str.replace(r'[^\w\s]','',regex = True)

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [24]:
tfidf = TfidfVectorizer(stop_words='english')

In [33]:
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [34]:
x_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34343 stored elements and shape (4457, 8151)>

In [35]:
df['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# Initialize and train
model = MultinomialNB()
model.fit(x_train_tfidf, y_train)

# Make predictions
predictions = model.predict(x_test_tfidf)

In [37]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[965   0]
 [ 36 114]]
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       965
        spam       1.00      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



In [41]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential

In [39]:
x_train_ann = x_train_tfidf.toarray()
x_test_ann = x_test_tfidf.toarray()
y_train_ann = y_train.map({'ham':0, 'spam':1}).values
y_test_ann = y_test.map({'ham':0, 'spam':1}).values

In [42]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=x_train_ann.shape[1]))
Dropout(0.5)
model.add(Dense(32, activation='relu'))
Dropout(0.3)
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [45]:
model.fit(x_train_ann, y_train_ann, epochs=30, batch_size=32,validation_split=0.1,
    class_weight={0: 1.0, 1: 6.0})

Epoch 1/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 2.1200e-05 - val_accuracy: 1.0000 - val_loss: 3.7852e-05
Epoch 2/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 1.8782e-05 - val_accuracy: 1.0000 - val_loss: 3.5563e-05
Epoch 3/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 1.6487e-05 - val_accuracy: 1.0000 - val_loss: 3.3846e-05
Epoch 4/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 1.4581e-05 - val_accuracy: 1.0000 - val_loss: 3.2087e-05
Epoch 5/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 1.3025e-05 - val_accuracy: 1.0000 - val_loss: 3.0604e-05
Epoch 6/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 1.1568e-05 - val_accuracy: 1.0000 - val_loss: 2.9570e-05
Epoch 7/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 1.0399e-05 - val_accuracy: 1.0000 - val_loss: 2.8349e-05
Epoch 8/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/s

In [47]:
def build_model(hp):
    model = Sequential()
    counter=0
    for i in range(10):
        if(counter==0):
            model.add(Dense(
                hp.Int('neuron '+str(i),min_value=1,max_value=10, ),
                activation = hp.Choice('activation',values=['relu','tanh','sigmoid']),
                input_dim=x_train_ann.shape[1] ))
            model.add(Dropout(hp.Choice('dropout '+str(i),values=[0.25,0.50,0.75,0.9])))
        else:
            model.add(Dense(
                hp.Int('neuron '+str(i),min_value=1,max_value=10, ),
                activation=hp.Choice('activation',values=['relu','rmsprop','tanh','sigmoid'])
            ))
            model.add(Dropout(hp.Choice('dropout '+str(i),values=[0.25,0.50,0.75,0.9])))
        counter+=1
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=hp.Choice('optimizer ',values=['rmsprop','adam','sgd','nadam','adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model


In [48]:
import keras_tuner as kt

In [49]:
import keras_tuner as kt

tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='models',
                        project_name='spam',)

In [50]:
tuner.search(x_train_ann, y_train_ann, epochs=5,validation_data=(x_test_ann, y_test_ann))

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.865470826625824

Best val_accuracy So Far: 0.865470826625824
Total elapsed time: 00h 00m 49s


In [51]:
tuner.get_best_hyperparameters()[0].values

{'neuron 0': 10,
 'activation': 'sigmoid',
 'dropout 0': 0.9,
 'neuron 1': 9,
 'dropout 1': 0.5,
 'neuron 2': 1,
 'dropout 2': 0.25,
 'neuron 3': 5,
 'dropout 3': 0.25,
 'neuron 4': 8,
 'dropout 4': 0.75,
 'neuron 5': 2,
 'dropout 5': 0.25,
 'neuron 6': 4,
 'dropout 6': 0.9,
 'neuron 7': 8,
 'dropout 7': 0.75,
 'neuron 8': 5,
 'dropout 8': 0.75,
 'neuron 9': 10,
 'dropout 9': 0.25,
 'optimizer ': 'sgd'}

In [53]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [57]:
tuned_model.fit(x_train_ann, y_train_ann, epochs=100,initial_epoch=5,validation_data=(x_test_ann, y_test_ann),class_weight={0: 1.0, 1: 6.0})

Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8322 - loss: 1.1564 - val_accuracy: 0.8655 - val_loss: 0.6599
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8661 - loss: 1.1572 - val_accuracy: 0.8655 - val_loss: 0.6667
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8178 - loss: 1.1567 - val_accuracy: 0.8655 - val_loss: 0.6503
Epoch 9/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8661 - loss: 1.1571 - val_accuracy: 0.8655 - val_loss: 0.6751
Epoch 10/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8028 - loss: 1.1574 - val_accuracy: 0.8655 - val_loss: 0.6629
Epoch 11/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8649 - loss: 1.1568 - val_accuracy: 0.8655 - val_loss: 0.6738
Epoch 12/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8553 - loss: 1.1574 - val_accuracy: 0.8655 - val_loss: 0.6621
Epoch 13/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8656 - loss: 1.1570 - val_

In [58]:
def build_model(hp):
    model = Sequential()

    # 1. Fewer Layers: For text, 2-3 dense layers are usually much better than 10.
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int('units_' + str(i), min_value=32, max_value=128, step=32),
            # 2. Use ReLU: It avoids the vanishing gradient problem.
            activation='relu'))
        # 3. Lower Dropout: Keep it between 0.2 and 0.5.
        model.add(Dropout(hp.Float('dropout_' + str(i), 0.2, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    # 4. Faster Optimizer: Adam or Nadam are better for sparse text data.
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [62]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='models',
                        project_name='spam1',)

In [63]:
tuner.search(x_train_ann, y_train_ann, epochs=5,validation_data=(x_test_ann, y_test_ann))

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.9847533702850342

Best val_accuracy So Far: 0.9847533702850342
Total elapsed time: 00h 01m 28s


In [65]:
tuned_model = tuner.get_best_models(num_models=1)[0]


In [67]:
from tensorflow.keras.callbacks import EarlyStopping

# Stop when val_loss hasn't improved for 3 epochs and keep the best version
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [68]:
tuned_model.fit(
    x_train_ann, y_train_ann,
    epochs=100,
    initial_epoch=5,
    validation_data=(x_test_ann, y_test_ann),
    class_weight={0: 1.0, 1: 6.0},
    callbacks=[early_stop])


Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 7.8281e-08 - val_accuracy: 0.9865 - val_loss: 0.2938
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 3.7844e-08 - val_accuracy: 0.9865 - val_loss: 0.2942
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 9.0827e-08 - val_accuracy: 0.9857 - val_loss: 0.2954
Epoch 9/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 3.1324e-08 - val_accuracy: 0.9857 - val_loss: 0.2958


In [69]:
from sklearn.metrics import confusion_matrix, classification_report

# Convert probabilities to binary 0 or 1
y_pred = (tuned_model.predict(x_test_ann) > 0.5).astype("int32")

print("--- Confusion Matrix ---")
print(confusion_matrix(y_test_ann, y_pred))
print("\n--- Classification Report ---")
print(classification_report(y_test_ann, y_pred))

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
--- Confusion Matrix ---
[[961   4]
 [ 11 139]]

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       965
           1       0.97      0.93      0.95       150

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Prediction: [[0.69523704]]


In [71]:
import numpy as np

# A variety of test cases
sample_emails = [
    "Hey! Are we still meeting at the library at 5? Let me know.", # Clean Ham
    "WINNER! You have been specially selected to receive a £1000 gift card. Call 0800 now to claim your prize!", # Obvious Spam
    "Your Netflix account is on hold. Please update your payment information at netflix-verify.com to avoid service interruption.", # Phishing Spam
    "The meeting agenda for tomorrow's board review is attached. Please review the financial projections before 9 AM.", # Formal Ham
    "URGENT: Your parcel is waiting for delivery. Please pay the £2.99 shipping fee at the link below.", # Common Delivery Spam
    "Can you send me the cash for the pizza? My Venmo is @PizzaGuy123." # Potential False Positive (contains "cash")
]

# Function to run the pipeline
def test_model(samples):
    print(f"{'PREDICTION':<12} | {'CONFIDENCE':<10} | {'TEXT SAMPLE'}")
    print("-" * 80)

    # Preprocess and predict
    # Ensure you are using the same 'tfidf' object from your training session!
    vec_samples = tfidf.transform(samples).toarray()
    probs = tuned_model.predict(vec_samples)

    for i, prob in enumerate(probs):
        label = "SPAM 🚩" if prob > 0.5 else "HAM ✅"
        confidence = prob[0] if prob > 0.5 else (1 - prob[0])
        print(f"{label:<12} | {confidence:>9.2%} | {samples[i][:60]}...")

test_model(sample_emails)

PREDICTION   | CONFIDENCE | TEXT SAMPLE
--------------------------------------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
HAM ✅        |   100.00% | Hey! Are we still meeting at the library at 5? Let me know....
SPAM 🚩       |   100.00% | WINNER! You have been specially selected to receive a £1000 ...
HAM ✅        |    78.31% | Your Netflix account is on hold. Please update your payment ...
HAM ✅        |   100.00% | The meeting agenda for tomorrow's board review is attached. ...
SPAM 🚩       |   100.00% | URGENT: Your parcel is waiting for delivery. Please pay the ...
HAM ✅        |   100.00% | Can you send me the cash for the pizza? My Venmo is @PizzaGu...
